<a href="https://colab.research.google.com/github/barkhaneum/Transformer_study/blob/main/chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -U datasets
!pip install evaluate
!pip install sacrebleu

from datasets import load_dataset
dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0")


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [2]:
sample = dataset['train'][1] # highlights, article, id
print(f"""기사 (500개 문자 발췌, 총 길이 : {len(sample['article'])} ) :""")
print(sample['article'][:500]) # 기사 내용 발췌
print(f'\n요약 ( 길이 : {len(sample["highlights"])}) :')
print(sample['highlights'])


기사 (500개 문자 발췌, 총 길이 : 3192 ) :
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has n

요약 ( 길이 : 180) :
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


In [4]:
sample_text = dataset['train'][1]['article'][:2000] # 기사 내용 발췌
summaries = {}

In [5]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")
nltk.download("punkt")

string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization.']

In [6]:
def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3]) # 텍스트중 맨처음 문장 세개를 선택

summaries["baseline"] = three_sentence_summary(sample_text)

In [10]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline("text-generation", model = "gpt2-xl")
gpt2_query = sample_text + "\nTl;DR:\n"

pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces = True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [15]:
pipe = pipeline("summarization", model = "t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

Device set to use cuda:0


In [16]:
pipe = pipeline("summarization", model = "facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [17]:
pipe = pipeline("summarization", model = "google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [18]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
  print(model_name.upper())
  print(summaries[model_name])
  print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .

BASELINE
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.

GPT2
The World Championships have come to a close and Usain Bolt has won another gold medal.
The Jamaican sprinter has now won eight gold medals at the world championships, equalling the record held by Carl Lewis, Michael Johnson and Allyson Felix.
Bolt is now third in the all-time list of World Championships gold medal

# 텍스트 생서 모델 평가

## 1. BLEU 평가

In [3]:

import evaluate
bleu_metric = evaluate.load("sacrebleu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np

bleu_metric.add(
    prediction = "the the the the the the", reference = ["the cat is on the mat"])
results = bleu_metric.compute(smooth_method = "floor", smooth_value = 0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient = "index", columns = ["Value"])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [10]:
bleu_metric.add(
    prediction = "the cat is on mat", reference = ["the cat is on the mat"])
results = bleu_metric.compute(smooth_method = "floor", smooth_value = 0)
results["precisions"] = [ np.round(p, 2) for p in results["precisions"] ]
pd.DataFrame.from_dict(results, orient = "index", columns = ["Value"])

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


## 2. ROUGE 평가

In [13]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rough1", "rouge2","rougeL", "rougeLsum"]

for model_name in summaries:
  rouge_metric.add(prediction == summaries[model_name], refernece = reference)
  score = rouge_metric.compute()
  rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
  records.append(rouge_dict)
pd.DataFrame.from_records(records, index = summaries.keys())

NameError: name 'dataset' is not defined

## CNN/DailyMail 데이터셋에서 PEGASUS 평가하기

In [ ]:
def evaluate_summaries_baseline(dataset, metric,
                                column_text = "article",
                                column_summary = "highlights"):
  summaries = [three_sentence_summary(text) for text in dataset[column_text]]
  metric.add_batch(predictions = summaries, references = dataset[column_summary])
  score = metric.compute()
  return score

In [17]:
test_sampled = dataset["test"].shuffle(seed = 42).select(range(1000))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mod.fmeasure) for  rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient = "index", columns ["baseline"]).T

SyntaxError: positional argument follows keyword argument (ipython-input-17-520880689.py, line 5)

In [20]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunk(list_of_elements, batch_size):
  """list_of_elements로부터 batch_size 크기의 청크를 연속적으로 생성함"""
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               bach_size = 16, device = device,
                               column_text = "article",
                               column_summary = "highlights"):
  article_batches = list(chunk(dataset[column_text], batch_size))
  target_batches = list(chunk(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total = len(article_batches)):

      inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                         padding = "max_length", return_tensors = "pt")

      summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                 attention_mask = inputs["attention_mask"].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length = 128)

      decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
                                            clean_up_tokenization_spaces = True)
                           for s in summaries]

      decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
      metric.add_batch(predictions = decoded_summaries, references = target_batch)

  score = metric.compute()
  return score



In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size = 8)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = ["pegasus"])

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

NameError: name 'test_sampled' is not defined

In [31]:
# samsum 데이터셋
dataset_samsum = load_dataset("samsum")
split_lenghts = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"분할 크기: {split_lengths}")
print(f"특성: {dataset_samsum['train'].column_names}")
print("\n대화 : ")
print(dataset_samsum["test"][0]["dialogue"])
print("\n요약 : ")
print(dataset_samsum["test"][0]["summary"])

FileNotFoundError: Couldn't find a dataset script at /content/samsum/samsum.py or any data file in the same directory. Couldn't find 'samsum' on the Hugging Face Hub either: FileNotFoundError: Dataset 'samsum' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.

In [26]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("요약 : ")
print(pipe_out[0]["summary_text"].repliace(" .<n>", ".\n"))

NameError: name 'pipe' is not defined

In [29]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model, tokenizer, batch_size = 8)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = ["pegasus"])

#  CNN/DailyMail데이터셋이 SAMSum과 다르기떄문에 좋은 결과가 나오진 않음
# 데이터셋을 훈련전 평가 파이프라인을 통해 훈련이 성공적인지 평가가 가능함
# 이 데이터셋에서 모델을 미세튜닝하면 ROUGE 점수가 향상되어야함

NameError: name 'dataset_samsum' is not defined

In [30]:
# samSUM데이터를 fine-tuning으로 훈련하기위한 데이터셋 분포 시각화
d_len = [len(tokenizer(x)["input_ids"]) for x in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer(x)["input_ids"]) for x in dataset_samsum["train"]["summary"]]

fig, axes = plt.subplots(1, 2, figsize = (10, 3.5), sharex = True)
axes[0].hist(d_len, bins = 20, color = "C0", edgecolor = "C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins = 20, color = "C0", edgecolor = "C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

NameError: name 'dataset_samsum' is not defined

In [2]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch["dialogue"], max_length = 1024, truncation = True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch["summary"], max_length = 128, truncation = True)

  return {"input_ids": input_encodings["input_ids"],
          "attention_mask" : input_encodings["attention_mask"],
          "labels":target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type = "torch", columns = columns)

NameError: name 'dataset_samsum' is not defined